# Prepare

In [1]:
import pathlib
import warnings

import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from ALLCools.mcds.MCDS import MCDS
from cemba_data.plot import cutoff_vs_cell_remain
from pybedtools import BedTool
import json

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/merge.py:18: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1829: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  'DataArray', pd.Series, pd.DataFrame, pd.Panel]:


## Parameters

In [2]:
cell_tidy_data_path = '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
cluster_col = 'SubType'
clustering_feature = 'chrom100k'
dask_distribute = True
in_memory = False
mcds_path_list = list(
    pathlib.Path('/home/hanliu/project/mouse_rostral_brain/dataset/').glob(
        '*mcds'))
exclude_chromosome = ['chrM', 'chrY']
black_list_path = '/home/hanliu/project/mouse_rostral_brain/misc/mm10-blacklist.v2.bed.gz'
min_feature_cov = 400
max_feature_cov = 3000
mc_type = 'CHN'

### Stable parameters

In [3]:
output_dir = 'Adata'
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## Load Data

In [4]:
if dask_distribute:
    from dask.distributed import Client
    client = Client(dashboard_address=':5555')

### Clustering results

In [5]:
cell_tidy_data = pd.read_msgpack(cell_tidy_data_path)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Raw Count Matrix (before filter)

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    mcds = MCDS.open(mcds_path_list,
                     use_cells=cell_tidy_data.index,
                     chunks={'cell': 1000})
if in_memory:
    mcds.load()

In [7]:
mcds

<xarray.MCDS>
Dimensions:              (cell: 103982, chrom100k: 27269, count_type: 2, gene: 55487, mc_type: 2)
Coordinates:
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * count_type           (count_type) object 'mc' 'cov'
    strand_type          <U4 'both'
    gene_chrom           (gene) object dask.array<shape=(55487,), chunksize=(55487,)>
    gene_start           (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
    gene_end             (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
  * cell                 (cell) object '10E_M_0' '10E_M_1' ... '9J_M_999'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 27265 27266 27267 27268
    chrom100k_chrom      (chrom100k) object dask.array<shape=(27269,), chunksize=(27269,)>
    chrom100k_bin_start  (chrom100k) int64 dask.array<shape=(27269,), chunksize=(27269,)>
    chrom100k_bin_end    (chrom100k) int64 dask.

## Filter Feature

### Remove some chromosome

In [8]:
mcds = mcds.sel({
    clustering_feature:
    ~mcds.coords[f'{clustering_feature}_chrom'].isin(exclude_chromosome)
})

### Remove blacklist

In [9]:
spetial_char = '_bin' if clustering_feature.startswith('chrom') else ''

feature_bed_df = pd.DataFrame([
    mcds.coords[f'{clustering_feature}_chrom'].to_pandas(),
    mcds.coords[f'{clustering_feature}{spetial_char}_start'].to_pandas(),
    mcds.coords[f'{clustering_feature}{spetial_char}_end'].to_pandas()
],
                              index=['chrom', 'start', 'end'],
                              columns=mcds.get_index(clustering_feature)).T
feature_bed = BedTool.from_dataframe(feature_bed_df)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1849: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1849: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/core/dataarray.py:1849: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}


In [10]:
black_list_bed = BedTool(black_list_path)
black_feature = feature_bed.intersect(black_list_bed, f=0.2, wa=True)
black_feature_index = black_feature.to_dataframe().set_index(
    ['chrom', 'start', 'end']).index
black_feature_id = pd.Index(feature_bed_df.reset_index()\
                       .set_index(['chrom', 'start', 'end'])\
                       .loc[black_feature_index][clustering_feature])

In [11]:
mcds = mcds.sel({
    clustering_feature:
    ~mcds.get_index(clustering_feature).isin(black_feature_id)
})

### Remove by mean cov

In [12]:
# # only calculate CHN
# fig, axes = cutoff_vs_cell_remain(mcds[f'{clustering_feature}_da']\
#                                   .sel(count_type='cov', mc_type='CHN')\
#                                   .mean(axis=0)\
#                                   .load())

In [13]:
before = mcds.coords[clustering_feature].size
mcds = mcds.filter_region_cov(dim=clustering_feature,
                              da=f'{clustering_feature}_da',
                              mc_type=mc_type,
                              min_cov=min_feature_cov,
                              max_cov=max_feature_cov)
after = mcds.coords[clustering_feature].size
print(
    f'Filter {clustering_feature} by [{min_feature_cov}, {max_feature_cov}], '
    f'filter rate {(after / before):.2f}. {after} features remained.')

Filter chrom100k by [400, 3000], filter rate 0.99. 24059 features remained.


In [14]:
mcds

<xarray.MCDS>
Dimensions:              (cell: 103982, chrom100k: 24059, count_type: 2, gene: 55487, mc_type: 2)
Coordinates:
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000064372.1'
  * count_type           (count_type) object 'mc' 'cov'
    strand_type          <U4 'both'
    gene_chrom           (gene) object dask.array<shape=(55487,), chunksize=(55487,)>
    gene_start           (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
    gene_end             (gene) int64 dask.array<shape=(55487,), chunksize=(55487,)>
  * cell                 (cell) object '10E_M_0' '10E_M_1' ... '9J_M_999'
  * chrom100k            (chrom100k) int64 30 31 32 33 ... 26335 26336 26337
    chrom100k_chrom      (chrom100k) object dask.array<shape=(24059,), chunksize=(24059,)>
    chrom100k_bin_start  (chrom100k) int64 dask.array<shape=(24059,), chunksize=(24059,)>
    chrom100k_bin_end    (chrom100k) int64 dask.arra

## Add rate

In [15]:
mcds.add_mc_rate(dim=clustering_feature, da=f'{clustering_feature}_da')

In [16]:
data = mcds[f'{clustering_feature}_da_rate'].sel(mc_type=mc_type).values

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('getitem-e9ca6f04b971462fa6d76f575aae276d', 53, 0)": ()}


## Make Adata

In [17]:
adata = anndata.AnnData(X=data,
                        obs=cell_tidy_data.reindex(mcds.get_index('cell')),
                        var=feature_bed_df.reindex(
                            mcds.get_index(clustering_feature)))

Transforming to str index.


In [18]:
adata

AnnData object with n_obs × n_vars = 103982 × 24059 
    obs: 'AllcPath', 'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate', 'CH_RateAdj', 'FinalReads', 'InputReads', 'MappedReads', 'Region', 'index_name', 'uid', 'BamFilteringRate', 'MappingRate', 'Pos96', 'Plate', 'Col96', 'Row96', 'Col384', 'Row384', 'FACS_Date', 'Slice', 'CellClass', 'l1-umap_0', 'l1-umap_1', 'l1-tsne_0', 'l1-tsne_1', 'MajorType', 'l2-umap_0', 'l2-umap_1', 'l2-tsne_0', 'l2-tsne_1', 'SubType', 'l3-umap_0', 'l3-umap_1', 'l3-tsne_0', 'l3-tsne_1', 'L1CellClass', 'class_tsne_0', 'class_tsne_1', 'class_umap_0', 'class_umap_1', 'Order', 'RegionName', 'MajorRegion', 'SubRegion', 'DetailRegion', 'PotentialOverlap (MMB)', 'Anterior (CCF coords)', 'Posterior (CCF coords)', 'MajorRegionColor', 'SubRegionColor', 'DissectionRegionColor', 'Replicate'
    var: 'chrom', 'start', 'end'

In [19]:
adata.write_h5ad(output_dir / 'cell_by_feature.cov_filter.rate.h5ad')

... storing 'Region' as categorical
... storing 'index_name' as categorical
... storing 'uid' as categorical
... storing 'Pos96' as categorical
... storing 'Plate' as categorical
... storing 'CellClass' as categorical
... storing 'MajorType' as categorical
... storing 'SubType' as categorical
... storing 'L1CellClass' as categorical
... storing 'RegionName' as categorical
... storing 'MajorRegion' as categorical
... storing 'SubRegion' as categorical
... storing 'DetailRegion' as categorical
... storing 'PotentialOverlap (MMB)' as categorical
... storing 'MajorRegionColor' as categorical
... storing 'SubRegionColor' as categorical
... storing 'DissectionRegionColor' as categorical
... storing 'Replicate' as categorical
... storing 'chrom' as categorical
... storing 'start' as categorical
... storing 'end' as categorical
